<a href="https://colab.research.google.com/github/juanalvarez96/KAGGLE_DATASCIENCE/blob/master/Kaggle_winners.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Libraries
%matplotlib inline
#PyTorch, of course
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
#We will need torchvision transforms for data augmentation

import torchvision.models as models
### utilities
# tool to print a nice summary of a network, similary to keras' summary
from torchsummary import summary

from torchvision import datasets, models, transforms

import time
import os
import copy

from __future__ import print_function
from __future__ import division

# library to do bash-like wildcard expansion
import glob
import matplotlib.pyplot as plt

# others
import numpy as np
import random
from PIL import Image
from IPython.display import display
from tqdm import tqdm_notebook
from torchvision import datasets
from google.colab import drive
drive.mount('/content/drive')

train_path = "/content/drive/My Drive/polytech-ds-2019/polytech-ds-2019/training"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Import Data and generate DataLoaders**
This process will be carried out using tbe torchvisison datasets set of function for importing data.\
Since the provided data is shuffled, we will organize each picture in folders. This implies, for every image from the training and validation folders, we will move it to a class folder.


1.   Create for the validation folder 10 folders following the structure: class0, class1, ..., class10
2.   Move every image from validation folder to its respective class folder. Move 01_xyz.jpg to class1, move 02_xyz.jpg to class2, ...
3. Repeat these two steps fro the training floder



In [0]:
transformations=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [0]:
# Needed to organize data for the classifier.
# Only do ONE TIME for the training
% cd /content/drive/My\ Drive/polytech-ds-2019/polytech-ds-2019/training
#! for i in `seq 0 10`; do mkdir class${i}; done
#! for i in `seq 0 10`; do mv ${i}_* class$i; done
! for i in `seq 0 10`; do ls -lh class${i}/ | wc -l; done

/content/drive/My Drive/polytech-ds-2019/polytech-ds-2019/training
995
430
1501
987
849
1326
441
281
856
1501
710


In [0]:
# Needed to organize data for the classifier.
# Only do ONE TIME for the validation
#! for i in `seq 0 10`; do mkdir ../validation/class${i}; done
#! for i in `seq 0 10`; do mv ../validation/${i}_* ../validation/class$i; done
! for i in `seq 0 10`; do ls -lh ../validation/class${i}/ | wc -l; done

365
145
502
329
327
451
148
100
348
504
233


In [0]:
# Load in each dataset and apply transformations using
# the torchvision.datasets as datasets library
% cd /content/drive/My\ Drive/polytech-ds-2019/polytech-ds-2019/training
data_set = datasets.ImageFolder(root="./", transform = transformations)
val_set = datasets.ImageFolder(root="../validation/", transform = transformations)

/content/drive/My Drive/polytech-ds-2019/polytech-ds-2019/training


In [0]:
data_loader = torch.utils.data.DataLoader(data_set, batch_size=32, shuffle=True)

In [0]:
% cd /content/drive/My\ Drive/polytech-ds-2019/polytech-ds-2019
data_dir = "./"

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "squeezenet"

# Number of classes in the dataset
num_classes = 11

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

/content/drive/My Drive/polytech-ds-2019/polytech-ds-2019


In [0]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [0]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [0]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Initializing Datasets and Dataloaders...")

Initializing Datasets and Dataloaders...


In [0]:
# Create training and validation datasets
data_set = datasets.ImageFolder(root="./training/", transform = transformations)
val_set = datasets.ImageFolder(root="./validation/", transform = transformations)
#ev_set = datasets.ImageFolder(root="./kaggle_evaluation/", transform = transformations)

image_datasets ={'train':data_set,'val':val_set}

# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}
#dataloader_eval = torch.utils.data.DataLoader(ev_set, batch_size=batch_size, shuffle=True, num_workers=4)

RuntimeError: ignored

In [0]:
! pwd

/content/drive/My Drive/polytech-ds-2019/polytech-ds-2019


In [0]:
# Send the model to GPU
#model_ft = model_ft.to(device)
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
model_ft = model_ft.to(device)
# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 classifier.1.weight
	 classifier.1.bias


In [0]:
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/14
----------
train Loss: 0.9292 Acc: 0.7034
val Loss: 0.7913 Acc: 0.7556

Epoch 1/14
----------
train Loss: 0.6896 Acc: 0.7777
val Loss: 0.7073 Acc: 0.7820

Epoch 2/14
----------
train Loss: 0.6302 Acc: 0.7969
val Loss: 0.6893 Acc: 0.7954

Epoch 3/14
----------
train Loss: 0.5979 Acc: 0.8032
val Loss: 0.6814 Acc: 0.7916

Epoch 4/14
----------
train Loss: 0.5767 Acc: 0.8118
val Loss: 0.6770 Acc: 0.7899

Epoch 5/14
----------
train Loss: 0.5486 Acc: 0.8234
val Loss: 0.6701 Acc: 0.8012

Epoch 6/14
----------
train Loss: 0.5420 Acc: 0.8213
val Loss: 0.6983 Acc: 0.7910

Epoch 7/14
----------
train Loss: 0.5281 Acc: 0.8271
val Loss: 0.6446 Acc: 0.8009

Epoch 8/14
----------
train Loss: 0.5230 Acc: 0.8291
val Loss: 0.7236 Acc: 0.7794

Epoch 9/14
----------
train Loss: 0.5192 Acc: 0.8289
val Loss: 0.6484 Acc: 0.8059

Epoch 10/14
----------
train Loss: 0.5040 Acc: 0.8342
val Loss: 0.6428 Acc: 0.8027

Epoch 11/14
----------
train Loss: 0.5065 Acc: 0.8351
val Loss: 0.6547 Acc: 0.8030

Ep

In [0]:
model_ft.eval

In [0]:
# Visualize image
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(data_loader))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)
class_names = data_set.classes
imshow(out, title=[class_names[x] for x in classes])
